In [1]:
# Import necessary libraries
import json
import math
from datetime import datetime
import time
from IPython.display import clear_output

# Function to read flight data from a JSON file
def read_flight_data(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Function to parse the flight data
def parse_data(flight_data):
    parsed_data = []
    for entry in flight_data:
        flight_id = entry['FlightID']
        timestamp = datetime.fromisoformat(entry['Timestamp'])
        lat = entry['Location']['Latitude']
        lon = entry['Location']['Longitude']
        speed = entry['Velocity']['Speed']
        heading = entry['Velocity']['Heading']
        parsed_data.append({
            'FlightID': flight_id,
            'Timestamp': timestamp,
            'Latitude': lat,
            'Longitude': lon,
            'Speed': speed,
            'Heading': heading
        })
    return parsed_data

# Function to calculate the distance between two coordinates using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    km = 6372.8 * c #6372.8 is the radius of earth in kilometers
    return km

# Function to calculate the time left to reach the destination
def time_left(distance, speed):
    if speed > 0:
        return distance / speed
    else:
        return float('inf')  # Infinite time if speed is zero

# Function to calculate the total cost
def calculate_cost(fuel_cost_per_km, speed, distance, delay_cost_per_minute, delay_time):
    time_left_in_minutes = time_left(distance, speed) * 60
    fuel_cost = fuel_cost_per_km * distance
    delay_cost = delay_cost_per_minute * delay_time
    total_cost = fuel_cost + delay_cost
    return total_cost

# Function to process the data and calculate costs iteratively
def process_flight_data(file_path, destination_lat, destination_lon, fuel_cost_per_km, delay_cost_per_minute):
    flight_data = read_flight_data(file_path)
    parsed_data = parse_data(flight_data)

    for i, entry in enumerate(parsed_data):
        if i == 0:
            continue

        prev_entry = parsed_data[i - 1]
        current_entry = entry

        distance_traveled = haversine(prev_entry['Latitude'], prev_entry['Longitude'], current_entry['Latitude'], current_entry['Longitude'])
        distance_to_destination = haversine(current_entry['Latitude'], current_entry['Longitude'], destination_lat, destination_lon)
        speed = current_entry['Speed']
        estimated_time_left = time_left(distance_to_destination, speed)
        
        # Calculate total cost for the current data point
        delay_time = estimated_time_left  # Assuming the delay time is the estimated time left
        total_cost = calculate_cost(fuel_cost_per_km, speed, distance_to_destination, delay_cost_per_minute, delay_time)
        
        # Clear the previous output
        clear_output(wait=True)
        
        # Display the results
        print(f"Timestamp: {current_entry['Timestamp']}")
        print(f"Distance Traveled: {distance_traveled:.2f} km")
        print(f"Distance to Destination: {distance_to_destination:.2f} km")
        print(f"Estimated Time Left: {estimated_time_left:.2f} hours")
        print(f"Total Cost: {total_cost:.2f} EUR")
        print("-----")
        
        # Simulate real-time delay (e.g., 1 second)
        time.sleep(1)

# Example parameters
destination_lat = 40.712776  # Example destination latitude
destination_lon = -74.005974  # Example destination longitude
fuel_cost_per_km = 0.5  # Example fuel cost per kilometer
delay_cost_per_minute = 70  # Example delay cost per minute

# Path to the flight data file
file_path = '/Users/tamar.alphaidze/Desktop/airline_dashboard/leafy_airline/cost_calculator/flight_telemetry_data.json'
#when better data is available mongodb will be populated and then the data will be retrieved from there
# Process the flight data and calculate costs iteratively
process_flight_data(file_path, destination_lat, destination_lon, fuel_cost_per_km, delay_cost_per_minute)


Timestamp: 2024-07-09 15:54:55.488105
Distance Traveled: 35.56 km
Distance to Destination: 8900.18 km
Estimated Time Left: 244.93 hours
Total Cost: 21594.85 EUR
-----
